Notebook to process the levodopa equivalent dose information from PPMI.

In [ ]:
import numpy as np
import pandas as pd

from calc_ledd import calc_ledd

### Demographics
Use preprocessed demographics. Enrolled patients have an enroll date that is not null.

In [ ]:
path = ### PUT PATH TO DATA HERE ###

In [ ]:
demo = pd.read_csv('demographics.csv',index_col=0) #note that this is the processed data created by 'Demographics_Data_Processing.ipynb
enrol_pat = demo[~demo.ENROLLDT.isnull()]

### Concomitant Medications

In [ ]:
con_med = pd.read_csv(path + 'Concomitant_Medications.csv')
con_med.head(4)

DISMED is a flag as to whether or not the medication is for PD. LEDD contains the levodopa equivalent daily dose (LEDD). 

Data we're intested in PATNO, EVENT_ID, LEDD, STARTDT, STARTEST, STOPDT, STOPEST, CMTRT, CMDOSE, CMDOSU, ONGOING for only PD medications and patients who are enrolled.

In [ ]:
cm = con_med[(con_med.DISMED==1) | ((con_med.DISMED.isnull()) & (~con_med.LEDD.isnull()))].merge(enrol_pat[['APPRDX','PATNO']], on=["PATNO"]) 

In [ ]:
PD_records = cm[cm.APPRDX==1].groupby('PATNO')

In [ ]:
counts_per_patient = PD_records.STARTDT.count().value_counts(sort=False)

In [ ]:
PD_pat = enrol_pat[enrol_pat.APPRDX==1.0]

Calculate LEDD for each patno date combination in the signature form

In [ ]:
signature = pd.read_csv(path + '/Signature_Form.csv')

In [ ]:
signature.head()

In [ ]:
ledd_processed = pd.DataFrame(columns=['PATNO','EVENT_ID','INFODT','LEDD'])

In [ ]:
cm.STARTDT = pd.to_datetime(cm.STARTDT)
cm.STOPDT = pd.to_datetime(cm.STOPDT)
for idx in signature.index:
    pn = signature.loc[idx].PATNO
    ei = signature.loc[idx].EVENT_ID
    idt = signature.loc[idx].INFODT
    ledd = calc_ledd(cm, pn, pd.to_datetime(idt))
    ledd_processed = ledd_processed.append({'PATNO':pn, 'EVENT_ID':ei, 'INFODT': idt, 'LEDD':ledd},
                                   ignore_index=True)

In [ ]:
ledd_processed.to_csv('ledd.csv')